In [38]:
%load_ext autoreload
%autoreload 2

In [39]:
from pathlib import Path
import datetime
import re
import win32com.client  #pip install pywin32
from generate_emails import *
from datetime import datetime, timedelta
import random
import json


## Generate random dates for the synthetic emails

In [13]:
# Set the range of the past week
start_date = datetime.now() - timedelta(days=7)
end_date = datetime.now()

# Generate random datetime objects within the range
random_dates = []
for _ in range(10):  # Generate 10 random datetime objects
    random_date = start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))
    random_dates.append(random_date)

# Print the generated random datetime objects
for date in random_dates:
    print(date)

2024-05-02 11:40:12.181096
2024-04-30 04:29:06.181096
2024-05-02 03:37:44.181096
2024-04-28 08:35:30.181096
2024-04-27 20:59:54.181096
2024-05-04 02:31:40.181096
2024-04-28 17:09:54.181096
2024-04-28 12:35:48.181096
2024-05-02 07:18:52.181096
2024-05-01 15:35:33.181096


In [ ]:
email_threads_topics = ["Developing Markets", "eFx", "US Equity Markets", "Fixed Income"]
authors = ["JP Morgan", "Goldman Sachs", "Morgan Stanley", "Bank of America"]
emails = ["jpmorgan@upc.edu", "goldmansachs@upc.edu", "morganstanley@upc.edu", "boa@upc.edu"]

recipients = ["hackupc-test@outlook.com"]

email_count = 0
num_emails = 2

message_list = []

In [45]:

for j in range(len(authors)):
    for topic in email_threads_topics:

        message = {}
        message['email_date'] = random_dates[email_count%len(random_dates)]
        message['sender'] = emails[j]
        message['recipient'] = recipients[0]
        message['subject'] = f"{authors[j]}: {topic} Update and Trading Ideas"
        message['email_id'] = email_count


        message['text'] = generate_email_body(topic, authors[j], previous_emails=None)

        #save_mbox(thread, f"./emails/{topic.replace(' ', '_').lower()}.txt")

        email_count += 1
        message_list.append(message)


input_dir = Path.cwd() / "Input"
with open(input_dir / "emails.json", "w") as f:
    json.dump(message_list, f, indent=4, default=str)

API key found in environment variables.


## Populate the emails in the inbox

In [49]:
# Get the hackupc-test@outlook.com account
account = None

outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")

for acc in namespace.Accounts:
    if acc.DisplayName == 'hackupc-test@outlook.com':
        account = acc
        break

if account is None:
    print("Hackupc account not found.")
else:
    # Iterate over the message_list and send emails
    for message in message_list:
        mail = outlook.CreateItem(0)  # Create a new email
        mail.Subject = message['subject']
        mail.Body = message['text']
        mail.To = message['recipient']
        mail.SendUsingAccount = account  # Set the account to use for sending
        mail.Send()


## Parse the emails from the Outlook Inbox

In [50]:


# Create output folder
output_dir = Path.cwd() / "Output"
output_dir.mkdir(parents=True, exist_ok=True)

# Connect to outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Connect to folder
#inbox = outlook.Folders('arvind.menon@epfl.ch').Folders('Inbox')
inbox = outlook.Folders('hackupc-test@outlook.com').Folders('Inbox')
# inbox = outlook.GetDefaultFolder(6)

# https://docs.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
# DeletedItems=3, Outbox=4, SentMail=5, Inbox=6, Drafts=16, FolderJunk=23

# Get messages
messages = inbox.Items

list_of_messages = []

for message in messages:
    subject = message.Subject
    body = message.body
    # attachments = message.Attachments
    recipients = message.Recipients
    sender = message.Sender
    date = message.SentOn
    entry_id = message.EntryID
    # Create separate folder for each message, exclude special characters and timestampe

    msg = dict()
    msg["subject"] = getattr(message, "Subject", "<UNKNOWN>")
    msg["recipient"] = [r.Name for r in message.Recipients]
    
    msg["email_date"] = getattr(message, "SentOn", "<UNKNOWN>")
    msg["email_id"] = getattr(message, "EntryID", "<UNKNOWN>")
    msg["sender"] = getattr(message, "Sender", "<UNKNOWN>")
    #msg["Size"] = getattr(message, "Size", "<UNKNOWN>")
    msg["text"] = getattr(message, "Body", "<UNKNOWN>")

    list_of_messages.append(msg)

import json
with open(output_dir / "emails.json", "w") as f:
    json.dump(list_of_messages, f, indent=4, default=str)

## Print the extracted elements from the inbox

In [52]:
def extract(count):
    """Get emails from outlook."""
    items = []
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.Folders('hackupc-test@outlook.com').Folders('Inbox')
    messages = inbox.Items
    message = messages.GetFirst()
    i = 0
    while message:
        try:
            msg = dict()
            msg["Subject"] = getattr(message, "Subject", "<UNKNOWN>")
            msg["Recipients"] = [r.Name for r in message.Recipients]
            
            msg["SentOn"] = getattr(message, "SentOn", "<UNKNOWN>")
            msg["EntryID"] = getattr(message, "EntryID", "<UNKNOWN>")
            msg["Sender"] = getattr(message, "Sender", "<UNKNOWN>")
            msg["Size"] = getattr(message, "Size", "<UNKNOWN>")
            msg["Body"] = getattr(message, "Body", "<UNKNOWN>")
            items.append(msg)
        except Exception as ex:
            print("Error processing mail", ex)
        i += 1
        if i < count:
            message = messages.GetNext()
        else:
            return items

    return items

def show_message(items):
    """Show the messages."""
    items.sort(key=lambda tup: tup["SentOn"])
    for i in items:
        print(i["SentOn"], i["Subject"], i["Recipients"])


def main():
    """Fetch and display top message."""
    items = extract(5)
    show_message(items)


if __name__ == "__main__":
    main()

2024-05-04 18:51:55+00:00 Bank of America: Fixed Income Update and Trading Ideas ['hackupc-test@outlook.com']
2024-05-04 19:05:32+00:00 JP Morgan: Developing Markets Update and Trading Ideas ['hackupc-test@outlook.com']
2024-05-04 19:05:33+00:00 JP Morgan: US Equity Markets Update and Trading Ideas ['hackupc-test@outlook.com']
2024-05-04 19:05:33+00:00 JP Morgan: eFx Update and Trading Ideas ['hackupc-test@outlook.com']
2024-05-04 19:05:34+00:00 JP Morgan: Fixed Income Update and Trading Ideas ['hackupc-test@outlook.com']
